## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


In [0]:
%sql
use country_club;
select name from facilities where membercost != 0;

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


In [0]:
%sql
use country_club;
select facid, name, membercost, monthlymaintenance from facilities
WHERE membercost < (0.2 * monthlymaintenance);


facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
2,Badminton Court,0.0,50
3,Table Tennis,0.0,10
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80
7,Snooker Table,0.0,15
8,Pool Table,0.0,15


In [0]:
%sql
use country_club;
select * from facilities where facid in (1,5);

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


In [0]:
%sql
use country_club;
select name, 
       monthlymaintenance,
       case when monthlymaintenance > 100 then 'expensive'
             else 'cheap' end as howCostly
from facilities;

name,monthlymaintenance,howCostly
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


In [0]:
%sql
use country_club;
select firstname, surname from members where joindate = (select max(joindate) from members);

firstname,surname
Darren,Smith


In [0]:
%sql
use country_club;
select distinct f.name as facility, concat(m.firstname, ' ', m.surname) as name from bookings as b
left join members as m on m.memid = b.memid
left join facilities as f on f.facid = b.facid
WHERE f.name LIKE '%Tennis Court%'
ORDER BY name;

facility,name
Tennis Court 2,Anne Baker
Tennis Court 1,Anne Baker
Tennis Court 2,Burton Tracy
Tennis Court 1,Burton Tracy
Tennis Court 1,Charles Owen
Tennis Court 2,Charles Owen
Tennis Court 2,Darren Smith
Tennis Court 2,David Farrell
Tennis Court 1,David Farrell
Tennis Court 2,David Jones


In [0]:
%sql
use country_club;
select b.bookId,
       concat(m.firstname, ' ', m.surname) AS memberName,
       f.name as facilityName,
       case when b.memid = 0
            then (b.slots * f.guestcost)
       else (b.slots * f.membercost) end as slotCost
       from bookings as b
left join facilities as f on b.facid = f.facid
left join members as m on b.memid = m.memid
WHERE date_trunc('day', b.starttime) = '2012-09-14'
AND (m.memid = 0 AND (b.slots * f.guestcost) > 30)
OR (m.memid != 0 AND (b.slots * f.membercost) > 30)
ORDER BY slotCost DESC;

bookId,memberName,facilityName,slotCost
2946,GUEST GUEST,Massage Room 2,320.0
2937,GUEST GUEST,Massage Room 1,160.0
2942,GUEST GUEST,Massage Room 1,160.0
2940,GUEST GUEST,Massage Room 1,160.0
2926,GUEST GUEST,Tennis Court 2,150.0
2920,GUEST GUEST,Tennis Court 1,75.0
2922,GUEST GUEST,Tennis Court 1,75.0
2925,GUEST GUEST,Tennis Court 2,75.0
2948,GUEST GUEST,Squash Court,70.0
530,Nancy Dare,Tennis Court 1,45.0


In [0]:
%sql
use country_club;
select memberName,
       facilityName,
       slotCost
       from (SELECT concat(m.firstname, ' ', m.surname) AS memberName,
                     f.name as facilityName,
                     case when b.memid = 0
                          then (b.slots * f.guestcost)
                     else (b.slots * f.membercost) end as slotCost
             FROM bookings as b
             left join facilities as f on b.facid = f.facid
             left join members as m on b.memid = m.memid
             WHERE date_trunc('day', b.starttime) = '2012-09-14'
             )
where slotCost > 30
ORDER BY slotCost DESC;

memberName,facilityName,slotCost
GUEST GUEST,Massage Room 2,320.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Tennis Court 2,150.0
GUEST GUEST,Tennis Court 2,75.0
GUEST GUEST,Tennis Court 1,75.0
GUEST GUEST,Tennis Court 1,75.0
GUEST GUEST,Squash Court,70.0
Jemima Farrell,Massage Room 1,39.6


In [0]:
%sql
use country_club;
select x.name, x.revenue from
(select f.facid,
       f.name,
       sum(case when b.memid = 0
                then (b.slots * f.guestcost)
           else (b.slots * f.membercost) END) AS revenue
from facilities as f
left join bookings as b on b.facid = f.facid
left join members as m on m.memid = b.memid
group by f.facid, f.name) as x
where x.revenue < 1000
order by x.facid
;

name,revenue
Table Tennis,180.0
Snooker Table,240.0
Pool Table,270.0
